In [55]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Mecab

In [56]:
mecab = Mecab()

In [2]:
urls = [
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108',
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790',
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362',
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191'
]

In [54]:
def get_news(url):
    headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.192 Safari/537.36"}
    res = requests.get(url, headers=headers)
    bs = BeautifulSoup(res.content, 'html.parser')
    title = bs.select_one('#articleTitle').get_text()
    content = bs.select("#articleBodyContents")[0].get_text()
    
    return content
    

In [63]:
def preprocess(content):
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    end_noise = re.search('[가-힣]{2,4}\s*\(\w+@\w+\.\w+\.*\w*\)', content)
    if end_noise:
        end_noise_index = end_noise.start()
        content = content[:end_noise_index]
    
    return ' '.join([token[0] for token in mecab.pos(content) if token[1] in ['NNG']])

In [62]:
' '.join([token[0] for token in mecab.pos(docs[0]) if token[1] in ['NNG']])

'과기 정통부 장관 참석 기념행사 투입 여종 데이터 구축 민간 외부 연계 체계 개방 강화 기자 국가 차원 빅 데이터 활용 시대 산업 창출 기존 산업 변화 혁신 장 센터 문 분야 데이터 경제 발전 정부 청사진 현실 구현 계획 과학 기술 정보 통신부 상공 회의소 데이터 생태 조성 혁신 성장 기반 마련 빅 데이터 플랫 폼 센터 출범식 행사 개최 과기 정통부 장관 국회 과학 기술 정보 방송 통신 위원회 위원장 참가 분야 센터 간 투입 이미지 데이터 데이터 활용 혁신 장 문재 정부 경제 성장 핵심 요소 대통령 데이터 활용 정보 보호 중요 강조 맥락 속 빅 데이터 센터 공공 민간 협업 활용 양질 데이터 생산 구축 플랫 폼 수집 분석 유통 역할 담당 과기 정통부 분야 플랫 폼 연계 기관 센터 구축 투입 계획 올해 규모 사업 추진 대상 분야 금융 카드 환경 수자원 공사 문화 문화 정보원 교통 교통 연구원 유통 소비 매일 방송 통신 중소기업 비즈 지역 경제 청 산림 임업 공모 빅 데이터 센터 선정 다음 달 공모 추가 선정 지원 운영 계획 데이터 생태계 혁신 기업 경쟁력 제고 역할 수행 주요 활용 전략 사례 빅 데이터 활용 신 新 시장 창출 방안 금융 플랫 폼 경우 소 상공 신용 평가 고도 금융 취약 계층 대상 금리 대출 절감 연간 신규 대출 창출 전망 유통 소비 중소기업 플랫 폼 소상 공인 중소기업 폐업 감소 문화 플랫 폼 문화 예술 관람 생활 체육 참여 방안 모색 의료비 절감 헬스 케어 기업 매출 향상 산업 육성 통신 산림 눈길 과기 정통부 제공 여종 데이터 구축 알고리즘 제공 센터 분야 데이터 부족 문제 해소 올해 시장 수요 여종 신규 데이터 생산 구축 사업 완료 여종 양질 데이터 생산 구축 시장 공급 계획 공공 민간 사이 데이터 파일 형식 호환 문제 해소 개방 표준 적용 품질 관리 기준 마련 운영 기업 실제 활용 가능 최신 데이터 확보 수개월 소요 문제점 개선 방안 추진 센터 플랫폼 연계 체계 민간 기반 활용 센터 축적 데이터 외부 개방 공유 최신 연속 확보 계획 센터 

In [64]:
docs = []
for url in urls:
    content = get_news(url)
    content = preprocess(content)
    docs.append(content)

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf = tfidf_vectorizer.fit_transform(docs)

In [69]:
feature_names = tfidf_vectorizer.get_feature_names()
keywords = tfidf.tocoo()

In [33]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)

In [36]:
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [70]:
for tfidf_per_doc in tfidf:
    keywords = tfidf_per_doc.tocoo()
    sorted_keywords = sort_keywords(keywords)
    print(extract_keywords(feature_names, sorted_keywords))
    print('\n')

[('데이터', 0.42673658365911715), ('플랫', 0.1846535329523627), ('계획', 0.1582744568163109), ('센터', 0.15142265871775124), ('정통부', 0.13189538068025908)]


[('전력', 0.29466079747085805), ('전력 데이터', 0.23572863797668642), ('데이터', 0.21527271911175336), ('대회', 0.1767964784825148), ('서비스', 0.15046253608880933)]


[('주행', 0.5311408811659298), ('자율 주행', 0.5090100111173493), ('자율', 0.5090100111173493), ('주행 센터', 0.13278522029148246), ('데이터', 0.12703686684133658)]


[('인구', 0.34197704612456714), ('발생', 0.3156711194996004), ('이동', 0.23675333962470033), ('지역', 0.22813884174736737), ('통계청', 0.2104474129997336)]




In [35]:
sorted_keywords = sort_keywords(keywords)


In [38]:
# 전체 문서에 대해서
print(extract_keywords(feature_names, sorted_keywords))

[('자율주행', 0.7037920051893569), ('인구', 0.40369651489859326), ('발생', 0.3669968317259939), ('데이터', 0.34917880509507887), ('전력', 0.3007183358906736)]


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = CountVectorizer()
DTM = count_vectorizer.fit_transform(docs)
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(DTM)

In [44]:
doc  = docs[0]

tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc]))
keywords = tf_idf_vector.tocoo()
sorted_keywords = sort_keywords(keywords)
print(extract_keywords(feature_names, sorted_keywords))

[('데이터', 0.349178805095079), ('계획이다', 0.18235202435332726), ('통해', 0.17252211662214476), ('데이터를', 0.1629501090443702), ('과기정통부', 0.1458816194826618)]


In [47]:
for i, doc in enumerate(docs):
    tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc]))
    keywords = tf_idf_vector.tocoo()
    sorted_keywords = sort_keywords(keywords)
    print("문서 {} :".format(i), extract_keywords(feature_names, sorted_keywords))

문서 0 : [('데이터', 0.349178805095079), ('계획이다', 0.18235202435332726), ('통해', 0.17252211662214476), ('데이터를', 0.1629501090443702), ('과기정통부', 0.1458816194826618)]
문서 1 : [('전력', 0.3007183358906737), ('한전', 0.2255387519180053), ('한국전력공사', 0.15035916794533685), ('학생', 0.15035916794533685), ('일반인', 0.15035916794533685)]
문서 2 : [('자율주행', 0.703792005189358), ('경기도', 0.28151680207574326), ('판교', 0.14075840103787163), ('실증단지', 0.14075840103787163), ('데이터', 0.11979235207480926)]
문서 3 : [('인구', 0.4036965148985938), ('발생', 0.3669968317259944), ('5월', 0.20254116225318478), ('이동', 0.1834984158629972), ('폭으로', 0.14679873269039775)]
